In [ ]:
import os
import shutil
import pandas as pd
import soundfile as sf
import librosa
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import warnings

warnings.filterwarnings('ignore')

# --- SABİTLER ---
SAMPLE_RATE = 16000
TRIM_DB = 25
OUTPUT_DIR = Path("/kaggle/working/processed_timit")

def find_dataset_root(start_dir="/kaggle/input"):
    """
    TIMIT klasörünü sistemde arayıp bulur.
    Path hatasını %100 çözer.
    """
    print("🔍 TIMIT veri seti aranıyor...")
    for root, dirs, files in os.walk(start_dir):
        # Eğer bir klasörde hem 'TRAIN' hem 'TEST' varsa orası doğru yerdir
        if "TRAIN" in dirs and "TEST" in dirs:
            found_path = Path(root)
            print(f"✅ Veri seti bulundu: {found_path}")
            return found_path
    
    return Path("/kaggle/input/timitd/data/lisa/data/timit/raw/TIMIT")

def get_gender_from_folder(folder_name, spkr_map):
    """
    Cinsiyeti bulmak için önce haritaya bakar, bulamazsa klasör adına bakar.
    TIMIT klasörleri genelde 'M' veya 'F' ile başlar (örn: FCJF0 -> Female).
    """
    # 1. Yöntem: Tam eşleşme (spkrinfo.txt)
    if folder_name in spkr_map:
        return spkr_map[folder_name]
    
    if len(folder_name) > 1 and folder_name[1:] in spkr_map:
        return spkr_map[folder_name[1:]]

    # 3. Yöntem: Klasör baş harfi (Fallback)
    if folder_name.upper().startswith("M"):
        return "M"
    elif folder_name.upper().startswith("F"):
        return "F"
    
    return None

def parse_spkrinfo(doc_path):
    spkr_map = {}
    try:
        # Büyük/küçük harf duyarlı arama
        info_files = list(doc_path.glob("spkrinfo.txt")) + list(doc_path.glob("SPKRINFO.TXT"))
        if not info_files:
            print("⚠️ spkrinfo.txt bulunamadı, klasör isimlerinden cinsiyet tahmini yapılacak.")
            return {}

        with open(info_files[0], 'r') as f:
            for line in f:
                parts = line.strip().split()
                if not parts or line.startswith(';'): continue
                # ID (örn: CJF0) -> Sex (M/F)
                spkr_map[parts[0]] = parts[1]
        return spkr_map
    except Exception as e:
        print(f"⚠️ Metadata okunamadı: {e}")
        return {}

def main():
    # 1. Veri setini bul
    ROOT_DIR = find_dataset_root()
    
    # 2. Metadata oku
    spkr_map = parse_spkrinfo(ROOT_DIR / "DOC")
    
    # İstatistikler
    stats = {"train": {"M": 0, "F": 0}, "test": {"M": 0, "F": 0}}
    
    # Çıktı klasörünü temizle/oluştur
    if OUTPUT_DIR.exists():
        shutil.rmtree(OUTPUT_DIR)
    
    print("\n🚀 İşleme Başlıyor...")
    
    for split in ["TRAIN", "TEST"]:
        split_dir = ROOT_DIR / split
        if not split_dir.exists():
            print(f"❌ {split} klasörü bulunamadı!")
            continue

        # Bölge klasörleri (DR1, DR2...)
        regions = [d for d in split_dir.iterdir() if d.is_dir()]
        
        for dr in tqdm(regions, desc=f"Processing {split}"):
            # Konuşmacı klasörleri
            speakers = [s for s in dr.iterdir() if s.is_dir()]
            
            for spkr in speakers:
                # Cinsiyet Tespiti
                gender = get_gender_from_folder(spkr.name, spkr_map)
                if not gender:
                    continue # Cinsiyet bilinemiyorsa atla
                
                # Dosyaları bul (WAV veya wav)
                # Büyük küçük harf duyarsız glob için list comprehension
                wav_files = [f for f in spkr.glob("*") if f.suffix.lower() == ".wav"]
                
                for wav_path in wav_files:
                    # SA Cümlesi Kontrolü (Dosya adında SA var mı?)
                    if "SA" in wav_path.name.upper().split(".")[0]: 
                        continue
                        
                    # İşleme
                    try:
                        y, sr = librosa.load(wav_path, sr=SAMPLE_RATE)
                        y_trim, _ = librosa.effects.trim(y, top_db=TRIM_DB)
                        
                        # Çok kısa dosyaları atla (<0.5sn)
                        if len(y_trim) < 0.5 * SAMPLE_RATE:
                            continue
                            
                        # Kaydet
                        label = "male" if gender == "M" else "female"
                        save_dir = OUTPUT_DIR / split.lower() / label
                        save_dir.mkdir(parents=True, exist_ok=True)
                        
                        out_name = f"{spkr.name}_{wav_path.name}"
                        sf.write(save_dir / out_name, y_trim, SAMPLE_RATE)
                        
                        stats[split.lower()][gender] += 1
                        
                    except Exception as e:
                        print(f"Hata: {wav_path.name} -> {e}")

    print("\n✅ İŞLEM TAMAMLANDI!")
    print("="*40)
    print(f"TRAIN -> Erkek: {stats['train']['M']}, Kadın: {stats['train']['F']}")
    print(f"TEST  -> Erkek: {stats['test']['M']}, Kadın: {stats['test']['F']}")
    print(f"Veriler: {OUTPUT_DIR}")
    print("="*40)

if __name__ == "__main__":
    main()

In [ ]:
import torch
import torchaudio
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import glob
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# --- KONFIGURASYON ---
DATA_ROOT = Path("/kaggle/working/processed_timit")
SAMPLE_RATE = 16000
DURATION = 3.0  
NUM_SAMPLES = int(SAMPLE_RATE * DURATION)
N_MELS = 80    

class TimitGenderDataset(Dataset):
    def __init__(self, subset="train", transform=None):
        self.subset_path = DATA_ROOT / subset
        self.files = []
        self.labels = []
        self.transform = transform 

        if not self.subset_path.exists():
            raise FileNotFoundError(f"Klasör yok: {self.subset_path}")

        print(f"📂 {subset.upper()} verisi taranıyor...")

        self.class_counts = [0, 0] # [Female, Male]
        
        for label_code, label_name in enumerate(["female", "male"]):
            class_dir = self.subset_path / label_name
            if not class_dir.exists(): continue
                
            class_files = list(class_dir.glob("*.wav")) + list(class_dir.glob("*.WAV"))
            
            count = len(class_files)
            print(f"   -> {label_name}: {count} dosya")
            
            self.files.extend(class_files)
            self.labels.extend([label_code] * count)
            self.class_counts[label_code] = count
            
        if len(self.files) == 0:
            raise ValueError(f"❌ {subset} setinde hiç dosya bulunamadı! (Uzantı hatası olabilir mi?)")

        # Spektrogram
        self.melspec = torchaudio.transforms.MelSpectrogram(
            sample_rate=SAMPLE_RATE, n_fft=1024, hop_length=160, n_mels=N_MELS
        )
        self.db_transform = torchaudio.transforms.AmplitudeToDB(stype="power", top_db=80)

    def __len__(self):
        return len(self.files)

    def _cut_or_pad(self, signal):
        length_signal = signal.shape[1]
        if length_signal < NUM_SAMPLES:
            num_repeats = int(NUM_SAMPLES / length_signal) + 1
            padded_signal = signal.repeat(1, num_repeats)
            signal = padded_signal[:, :NUM_SAMPLES]
        elif length_signal > NUM_SAMPLES:
            start = random.randint(0, length_signal - NUM_SAMPLES)
            signal = signal[:, start:start + NUM_SAMPLES]
        return signal

    def __getitem__(self, index):
        file_path = self.files[index]
        label = self.labels[index]
        
        signal, sr = torchaudio.load(file_path)
        if sr != SAMPLE_RATE:
            signal = torchaudio.transforms.Resample(sr, SAMPLE_RATE)(signal)
        signal = self._cut_or_pad(signal)

        spec = self.melspec(signal)
        spec = self.db_transform(spec) 

        if self.transform:
            spec = self.transform(spec)
            
        mean = spec.mean()
        std = spec.std()
        spec = (spec - mean) / (std + 1e-6)

        return spec, label

# --- AUGMENTATION ---
train_transforms = nn.Sequential(
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35)
)

def get_dataloaders(batch_size=32):
    print("\n--- DATASET OLUŞTURULUYOR ---")
    train_ds = TimitGenderDataset("train", transform=train_transforms)
    test_ds = TimitGenderDataset("test", transform=None)
    
    # Dengesizlik Çözümü
    counts = train_ds.class_counts
    # Bölme hatasını önle (if count > 0)
    weights = [1.0/c if c > 0 else 0 for c in counts]
    
    sample_weights = [weights[label] for label in train_ds.labels]
    
    sampler = WeightedRandomSampler(
        weights=torch.DoubleTensor(sample_weights),
        num_samples=len(sample_weights),
        replacement=True
    )
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, sampler=sampler, num_workers=2)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)
    
    return train_loader, test_loader

def visualize_sample():
    try:
        train_loader, _ = get_dataloaders(batch_size=4)
        specs, labels = next(iter(train_loader))
        
        print(f"\n✅ BAŞARILI! Batch Şekli: {specs.shape}")
        
        plt.figure(figsize=(10, 4))
        plt.imshow(specs[0].squeeze().numpy(), cmap='inferno', origin='lower')
        plt.title(f"Log-Mel Spektrogram (Label: {'Erkek' if labels[0]==1 else 'Kadın'})")
        plt.colorbar(format='%+2.0f dB')
        plt.show()
        
    except Exception as e:
        print(f"\n❌ HATA: {e}")

if __name__ == "__main__":
    visualize_sample()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SEModule(nn.Module):
    def __init__(self, channels, reduction=8):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, t = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y

class Res2NetBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, scale=8):
        super(Res2NetBlock, self).__init__()
        self.scale = scale
        self.width = out_channels // scale
        self.nums = scale if scale == 1 else scale - 1

        self.convs = nn.ModuleList([
            nn.Conv1d(self.width, self.width, kernel_size, dilation=dilation, padding=(kernel_size - 1) * dilation // 2)
            for _ in range(self.nums)
        ])
        self.bns = nn.ModuleList([nn.BatchNorm1d(self.width) for _ in range(self.nums)])
        self.se = SEModule(out_channels)

    def forward(self, x):
        out = []
        spx = torch.split(x, self.width, 1)
        for i in range(self.nums):
            if i == 0:
                sp = spx[i]
            else:
                sp = sp + spx[i]
            sp = self.convs[i](sp)
            sp = self.bns[i](sp)
            sp = F.relu(sp)
            out.append(sp)
        
        if self.scale != 1:
            out.append(spx[self.nums])
            
        out = torch.cat(out, dim=1)
        out = self.se(out)
        return out

class AttentiveStatsPooling(nn.Module):

    def __init__(self, in_dim, attention_channels=128):
        super().__init__()
        self.conv = nn.Conv1d(in_dim, attention_channels, kernel_size=1)
        self.conv2 = nn.Conv1d(attention_channels, in_dim, kernel_size=1)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        # x: (Batch, Channels, Time)
        
        # 1. Dikkat Skorlarını Hesapla
        attn = self.conv(x)
        attn = self.tanh(attn)
        attn = self.conv2(attn)
        attn = self.softmax(attn) # (Batch, Channels, Time)

        # 2. Ağırlıklı Ortalama (Mean)
        # sum(x * w) -> (Batch, Channels)
        mean = torch.sum(x * attn, dim=2)
        
        # 3. Ağırlıklı Standart Sapma (Std)
        # Varyans = sum(w * (x - mean)^2)
        # Yayma (Broadcasting): (B, C, T) - (B, C, 1) = (B, C, T)
        residuals = (x - mean.unsqueeze(2)).pow(2)
        weighted_residuals = torch.sum(residuals * attn, dim=2) # (Batch, Channels)
        
        std = torch.sqrt(weighted_residuals.clamp(min=1e-9))
        
        # 4. Birleştir
        return torch.cat([mean, std], dim=1)

class ECAPA_TDNN_Gender(nn.Module):
    def __init__(self, input_channels=80, channels=512, emb_dim=192, num_classes=2):
        super(ECAPA_TDNN_Gender, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(input_channels, channels, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(channels),
            nn.ReLU(inplace=True)
        )

        self.layer2 = Res2NetBlock(channels, channels, kernel_size=3, dilation=2, scale=8)
        self.layer3 = Res2NetBlock(channels, channels, kernel_size=3, dilation=3, scale=8)
        self.layer4 = Res2NetBlock(channels, channels, kernel_size=3, dilation=4, scale=8)

        self.layer5 = nn.Sequential(
            nn.Conv1d(3 * channels, 1536, kernel_size=1),
            nn.BatchNorm1d(1536),
            nn.ReLU(inplace=True)
        )

        self.pooling = AttentiveStatsPooling(1536)
        self.bn_pooling = nn.BatchNorm1d(1536 * 2)

        self.fc = nn.Linear(1536 * 2, emb_dim)
        self.bn_fc = nn.BatchNorm1d(emb_dim)
        self.output_layer = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        # x shape: (Batch, 1, Freq, Time) -> (Batch, Freq, Time)
        if x.dim() == 4:
            x = x.squeeze(1) 

        out1 = self.layer1(x)
        out2 = self.layer2(out1) + out1
        out3 = self.layer3(out2) + out2
        out4 = self.layer4(out3) + out3

        out = torch.cat([out2, out3, out4], dim=1)
        out = self.layer5(out)
        
        out = self.pooling(out)
        out = self.bn_pooling(out)
        
        out = self.fc(out)
        out = self.bn_fc(out)
        out = F.relu(out)
        
        logits = self.output_layer(out)
        return logits

# --- TEST BLOĞU ---
if __name__ == "__main__":
    print("🏗️ ECAPA-TDNN Modeli (V2) İnşa Ediliyor...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = ECAPA_TDNN_Gender(num_classes=2).to(device)
    
    dummy_input = torch.randn(4, 1, 80, 301).to(device)
    
    try:
        output = model(dummy_input)
        print(f"✅ GİRİŞ: {dummy_input.shape}")
        print(f"✅ ÇIKIŞ: {output.shape}") 
        
        if output.shape == (4, 2):
            print("🚀 Model Mimarisı DOĞRULANDI! Eğitime hazırız.")
        else:
            print("❌ Boyut hatası devam ediyor.")
            
    except Exception as e:
        print(f"❌ HATA: {e}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
import copy
from tqdm.auto import tqdm



# --- AYARLAR (Rapordaki Tablo 1'e Göre) ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0.01
EPOCHS = 15 # Hızlı sonuç için 15 yeterli (TIMIT küçük bir veri seti)
BATCH_SIZE = 32

def train_model():
    print(f"🔥 Eğitim Başlıyor... (Cihaz: {DEVICE})")
    
    # 1. Veri Yükleyicileri Al
    train_loader, test_loader = get_dataloaders(batch_size=BATCH_SIZE)
    print(f"   -> Eğitim Adımı Sayısı: {len(train_loader)}")
    
    # 2. Modeli Başlat
    model = ECAPA_TDNN_Gender(num_classes=2).to(DEVICE)
    
    # 3. Kayıp Fonksiyonu, Optimizer ve Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
    
    # Takip Değişkenleri
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    history = {'train_loss': [], 'val_acc': []}
    
    start_time = time.time()

    # --- EĞİTİM DÖNGÜSÜ ---
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        print("-" * 10)
        
        # --- TRAIN ---
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Tqdm barı
        pbar = tqdm(train_loader, desc="Eğitiliyor", leave=False)
        
        for inputs, labels in pbar:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            
            # Sıfırla -> İleri -> Hata -> Geri -> Güncelle
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            # İstatistikler
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += torch.sum(preds == labels.data)
            total_train += labels.size(0)
            
            # Bar güncelle
            pbar.set_postfix({'loss': loss.item()})
            
        epoch_loss = running_loss / len(train_loader.dataset)
        train_acc = correct_train.double() / total_train
        
        # --- VALIDATION (TEST) ---
        model.eval()
        running_corrects = 0
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)
                
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                
                running_corrects += torch.sum(preds == labels.data)
        
        epoch_acc = running_corrects.double() / len(test_loader.dataset)
        
        history['train_loss'].append(epoch_loss)
        history['val_acc'].append(epoch_acc.item())
        
        print(f"Loss: {epoch_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {epoch_acc:.4f}")
        
        # Scheduler Adımı
        scheduler.step()
        
        # En iyi modeli kaydet
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), "best_model_ecapa.pth")
            print("✅ Yeni En İyi Model Kaydedildi!")

    time_elapsed = time.time() - start_time
    print(f"\n🏁 Eğitim Tamamlandı! Süre: {time_elapsed // 60:.0f}dk {time_elapsed % 60:.0f}sn")
    print(f"🏆 En Yüksek Test Doğruluğu (Val Acc): {best_acc:.4f}")
    
    # En iyi ağırlıkları yükle
    model.load_state_dict(best_model_wts)
    return model, history

if __name__ == "__main__":
    # Eğer önceki hücrelerden fonksiyonlar hafızadaysa doğrudan çalışır
    trained_model, history = train_model()

In [ ]:
import torch
import torchaudio
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from pathlib import Path


class GenderPredictor:
    def __init__(self, model_path, device=None):
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        
        # Modeli Başlat
        self.model = ECAPA_TDNN_Gender(num_classes=2).to(self.device)
        
        # Ağırlıkları Yükle (Hata kontrolü ile)
        if not Path(model_path).exists():
            raise FileNotFoundError(f"Model dosyası bulunamadı: {model_path}")
            
        self.model.load_state_dict(torch.load(model_path, map_location=self.device))
        self.model.eval()
        
        self.sample_rate = 16000
        self.duration = 3.0
        self.num_samples = int(self.sample_rate * self.duration)
        
        # Dönüşüm Araçları
        self.melspec = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.sample_rate, n_fft=1024, hop_length=160, n_mels=80
        ).to(self.device)
        self.db_transform = torchaudio.transforms.AmplitudeToDB(stype="power", top_db=80).to(self.device)

    def preprocess(self, file_path):
        signal, sr = torchaudio.load(file_path)
        signal = signal.to(self.device)
        
        if sr != self.sample_rate:
            resampler = torchaudio.transforms.Resample(sr, self.sample_rate).to(self.device)
            signal = resampler(signal)
            
        length = signal.shape[1]
        if length < self.num_samples:
            repeats = int(self.num_samples / length) + 1
            signal = signal.repeat(1, repeats)
            signal = signal[:, :self.num_samples]
        else:
            # Ortadan kes
            start = (length - self.num_samples) // 2
            signal = signal[:, start:start + self.num_samples]
            
        spec = self.melspec(signal)
        spec = self.db_transform(spec)
        
        mean = spec.mean()
        std = spec.std()
        spec = (spec - mean) / (std + 1e-6)
        
        return spec.unsqueeze(0)

    def predict(self, file_path):
        spec = self.preprocess(file_path)
        with torch.no_grad():
            logits = self.model(spec)
            probs = F.softmax(logits, dim=1)
            
        score, pred_idx = torch.max(probs, 1)
        label = "Erkek" if pred_idx.item() == 1 else "Kadın"
        return label, score.item()

def evaluate_on_test_set(predictor):
    print("🔍 Test Seti Analiz Ediliyor...")
    
    test_root = Path("/kaggle/working/processed_timit/test")
    y_true = []
    y_pred = []
    files = []
    
    # Her iki sınıfı gez
    for label_code, label_name in enumerate(["female", "male"]):
        folder = test_root / label_name
        
        wavs = list(folder.glob("*.wav")) + list(folder.glob("*.WAV"))
        
        print(f"   -> {label_name}: {len(wavs)} dosya test ediliyor...")
        
        for wav in wavs:
            try:
                pred_label, conf = predictor.predict(str(wav))
                pred_code = 1 if pred_label == "Erkek" else 0
                
                y_true.append(label_code)
                y_pred.append(pred_code)
                files.append(wav.name)
            except Exception as e:
                print(f"Hata ({wav.name}): {e}")

    # Grafik Çizimi
    if len(y_true) > 0:
        cm = confusion_matrix(y_true, y_pred)
        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=['Kadın (Tahmin)', 'Erkek (Tahmin)'],
                    yticklabels=['Kadın (Gerçek)', 'Erkek (Gerçek)'])
        plt.title('TIMIT Sonuç Matrisi')
        plt.ylabel('Gerçek')
        plt.xlabel('Tahmin')
        plt.show()
        
        print("\n" + "="*40)
        print("📊 DETAYLI PERFORMANS RAPORU")
        print("="*40)
        print(classification_report(y_true, y_pred, target_names=['Kadın', 'Erkek'], digits=4))
    else:
        print("❌ HATA: Test edilecek dosya bulunamadı!")

if __name__ == "__main__":
    model_path = "best_model_ecapa.pth"
    if Path(model_path).exists():
        predictor = GenderPredictor(model_path)
        
        test_dir = Path("/kaggle/working/processed_timit/test/male")
        # Hem küçük hem büyük harf ara
        all_files = list(test_dir.glob("*.wav")) + list(test_dir.glob("*.WAV"))
        
        if all_files:
            test_file = all_files[0]
            label, score = predictor.predict(str(test_file))
            print(f"\n🧪 Tekil Test: {test_file.name}")
            print(f"   Sonuç: {label} (Güven: %{score*100:.2f})")
            
            # Tam Analiz
            evaluate_on_test_set(predictor)
        else:
            print("❌ Test klasöründe hiç dosya yok! (Yol hatası?)")
            import os
            os.system("ls -R /kaggle/working/processed_timit/test")
    else:
        print("⚠️ Model dosyası bulunamadı. Lütfen önce eğitimi tamamlayın.")